# Explore Integrated People with Air Theme 1
Look at ANOVAs and descriptive statistics for joined data

## Description of Program
- program:    ip1_3bv1_peopleair
- task:       Explore integrated people and air data
- Version:    2025-12-19
- project:    Southeast Texas Urban Integrated Field Lab
- funding:	  DOE
- author:     Nathanael Rosenheim

## Step 0: Good Housekeeping

In [9]:
# 1. Import all packages
import pandas as pd     # For obtaining and cleaning tabular data
import geopandas as gpd # For obtaining and cleaning spatial data
import matplotlib.pyplot as plt # For plotting
import contextily as ctx # For adding basemaps
import os # For saving output to path
import zipfile # For handling zip files
import io # For handling in-memory data
import requests # For downloading data
import rasterio # For reading geotiff files

In [10]:
# 2. Check versions
import sys
print("Python Version     ", sys.version)
print("geopandas version: ", gpd.__version__)

Python Version      3.13.9 | packaged by conda-forge | (main, Oct 22 2025, 23:12:41) [MSC v.1944 64 bit (AMD64)]
geopandas version:  1.1.1


In [11]:
# 3. Check working directory
# Get information on current working directory (getcwd)
os.getcwd()

'c:\\Users\\nathanael99\\MyProjects\\GitHub\\integrate_people_theme1_cookbook'

In [12]:
#4. Store Program Name for output files to have the same name
programname = "ip1_3bv1_peopleair"
# Make directory to save output
if not os.path.exists(programname):
    os.mkdir(programname)

# Step 1: Obtain Data
Obtain data from clean data step

In [13]:
# read in cleaned air data from ip1_2av1_convertairtocsv
source_programname = "ip1_2av1_joinpeopleair"
source_filename = f"{source_programname}.csv"

hua_ari_df = pd.read_csv(os.path.join(source_programname, source_filename))


C:\Users\nathanael99\AppData\Local\Temp\ipykernel_16940\2341540361.py:5: DtypeWarning: Columns (28,34,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  hua_ari_df = pd.read_csv(os.path.join(source_programname, source_filename))


In [14]:
hua_ari_df.head(1)

,huid,Block2020,blockid,bgid,tractid,FIPScounty,numprec,ownershp,race,hispan,...,uifl_4km_Xylenes_p98,uifl_4km_Xylenes_mean,uifl_4km_Xylenes_p75,uifl_4km_Xylenes_p25,uifl_4km_Xylenes_p90,uifl_4km_Xylenes_p95,Household Income Group,Tenure Status,Low Income Renter Status,Race Ethnicity
0,B481990302002008H002,481990302002008,481990302002008,481990302002,48199030200,48199.0,0.0,NaN,NaN,NaN,...,0.061573,0.011893,0.015375,0.001948,0.029066,0.04408,NaN,NaN,NaN,NaN


In [15]:
# print all columns in hua_ari_df
for col in hua_ari_df.columns:
    print(col)

huid
Block2020
blockid
bgid
tractid
FIPScounty
numprec
ownershp
race
hispan
family
vacancy
gqtype
incomegroup
hhinc
randincome
poverty
BLOCKID20_str
huicounter1
huicounter2
huicounter3
ownershp1
ownershp2
ownershp3
strctid
strctid_flagsetrm
strctid_Block2020_flagsetrm
addrptid
fd_id_bid
huestimate
huicounter_addpt
placeNAME20
x
y
occtype
geometry
Site_ID
Site_Name
index__1km
air_grid_id_left
row_idx_left
col_idx_left
uifl_1km_Benzene_p75
uifl_1km_Benzene_p100
uifl_1km_Benzene_p95
uifl_1km_Benzene_p50
uifl_1km_Benzene_p25
uifl_1km_Benzene_p98
uifl_1km_Benzene_p90
uifl_1km_Benzene_mean
uifl_1km_Benzene_p99
uifl_1km_Acetonitrile_p90
uifl_1km_Acetonitrile_p98
uifl_1km_Acetonitrile_mean
uifl_1km_Acetonitrile_p50
uifl_1km_Acetonitrile_p75
uifl_1km_Acetonitrile_p99
uifl_1km_Acetonitrile_p25
uifl_1km_Acetonitrile_p95
uifl_1km_Acetonitrile_p100
uifl_1km_1,3-Butadiene_p100
uifl_1km_1,3-Butadiene_p90
uifl_1km_1,3-Butadiene_p98
uifl_1km_1,3-Butadiene_p95
uifl_1km_1,3-Butadiene_p75
uifl_1km_1,3-But

# Step 2: Clean Data

# Step 3: Explore Data

In [ ]:
# Run ANOVA test on hua_air_gdf_with_demographics using the ownership variable 
# 1 = owner occupied and 2 = renter occupied

import scipy.stats as stats
import numpy as np


    
# Check if we have air pollution data
pollution_col = f'uifl_1km_Benzene_p90'

    
# Prepare data for ANOVA - exclude zeros since they are outside the model domain
analysis_df = hua_air_gdf_with_demographics[[pollution_col, 'ownershp']].dropna()
analysis_df = analysis_df[analysis_df[pollution_col] > 0]

print(f"\n=== DATA SUMMARY ===")
print(f"Total records with valid pollution and ownership data (excluding zeros): {len(analysis_df)}")
print(f"Note: Zero values excluded as they represent areas outside the pollution model domain")

print(f"\n" + "="*60)
print(f"ANOVA ANALYSIS - NON-ZERO POLLUTION VALUES ONLY")
print(f"="*60)

print(f"Sample size for ANOVA: {len(analysis_df)}")
print(f"Owner-occupied units (1): {(analysis_df['ownershp'] == 1).sum()}")
print(f"Renter-occupied units (2): {(analysis_df['ownershp'] == 2).sum()}")

if len(analysis_df) > 0 and analysis_df['ownershp'].nunique() >= 2:
    # Split data by ownership type
    owner_occupied = analysis_df[analysis_df['ownershp'] == 1][pollution_col]
    renter_occupied = analysis_df[analysis_df['ownershp'] == 2][pollution_col]
    
    if len(owner_occupied) > 0 and len(renter_occupied) > 0:
        # Calculate descriptive statistics
        print(f"\n--- Descriptive Statistics for {first_pollutant} by Ownership ---")
        print(f"Owner-Occupied (n={len(owner_occupied)}):")
        print(f"  Mean: {owner_occupied.mean():.4f} ppb")
        print(f"  Std:  {owner_occupied.std():.4f} ppb")
        print(f"  Min:  {owner_occupied.min():.4f} ppb")
        print(f"  Max:  {owner_occupied.max():.4f} ppb")
        
        print(f"Renter-Occupied (n={len(renter_occupied)}):")
        print(f"  Mean: {renter_occupied.mean():.4f} ppb")
        print(f"  Std:  {renter_occupied.std():.4f} ppb")
        print(f"  Min:  {renter_occupied.min():.4f} ppb")
        print(f"  Max:  {renter_occupied.max():.4f} ppb")
        
        # Run one-way ANOVA
        print(f"\n--- One-Way ANOVA Results ---")
        f_statistic, p_value = stats.f_oneway(owner_occupied, renter_occupied)
        
        print(f"F-statistic: {f_statistic:.4f}")
        print(f"p-value: {p_value:.6f}")
        
        # Interpretation
        alpha = 0.05
        if p_value < alpha:
            print(f"Result: Significant difference (p < {alpha})")
            print(f"There IS a statistically significant difference in {first_pollutant} exposure")
            print(f"between owner-occupied and renter-occupied housing units.")
        else:
            print(f"Result: No significant difference (p >= {alpha})")
            print(f"There is NO statistically significant difference in {first_pollutant} exposure")
            print(f"between owner-occupied and renter-occupied housing units.")
        
        # Effect size (Cohen's d)
        pooled_std = np.sqrt(((len(owner_occupied) - 1) * owner_occupied.var() + 
                            (len(renter_occupied) - 1) * renter_occupied.var()) / 
                            (len(owner_occupied) + len(renter_occupied) - 2))
        
        if pooled_std > 0:
            cohens_d = (owner_occupied.mean() - renter_occupied.mean()) / pooled_std
            print(f"Cohen's d (effect size): {cohens_d:.4f}")
            
            # Interpretation of effect size
            if abs(cohens_d) < 0.2:
                effect_size_interpretation = "negligible"
            elif abs(cohens_d) < 0.5:
                effect_size_interpretation = "small"
            elif abs(cohens_d) < 0.8:
                effect_size_interpretation = "medium"
            else:
                effect_size_interpretation = "large"
            
            print(f"Effect size interpretation: {effect_size_interpretation}")
            
            if cohens_d > 0:
                print("Direction: Owner-occupied units have HIGHER pollution exposure")
            else:
                print("Direction: Renter-occupied units have HIGHER pollution exposure")
        else:
            print("Cannot calculate Cohen's d (pooled standard deviation is 0)")
        
        # Additional test: Levene's test for equal variances
        levene_stat, levene_p = stats.levene(owner_occupied, renter_occupied)
        print(f"\n--- Levene's Test for Equal Variances ---")
        print(f"Levene's statistic: {levene_stat:.4f}")
        print(f"p-value: {levene_p:.6f}")
        if levene_p < 0.05:
            print("Warning: Variances are significantly different (ANOVA assumption violated)")
            # Run Welch's t-test as alternative
            welch_stat, welch_p = stats.ttest_ind(owner_occupied, renter_occupied, equal_var=False)
            print(f"\nWelch's t-test (unequal variances):")
            print(f"t-statistic: {welch_stat:.4f}")
            print(f"p-value: {welch_p:.6f}")
        else:
            print("Variances are not significantly different (ANOVA assumption met)")
    else:
        print(f"Insufficient data: Owner-occupied n={len(owner_occupied)}, Renter-occupied n={len(renter_occupied)}")


#### Look at summary population data

In [ ]:
# Add race ethnicity categories
hua_tfsites_gdf_addrace = PopResultsTable.add_race_ethnicity_to_pop_df(hua_with_grid)

In [ ]:
PopResultsTable.pop_results_table(
                  input_df = hua_tfsites_gdf_addrace, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = 'Southeast Texas',
                  when = '2020',
                  row_index = "Race Ethnicity",
                  col_index = 'Tenure Status')

In [ ]:
# select place name = Beaumont
condition = hua_tfsites_gdf_addrace['placeNAME20'] == 'Beaumont'
beaumont_gdf = hua_tfsites_gdf_addrace[condition]
PopResultsTable.pop_results_table(
                  input_df = beaumont_gdf, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = 'Beaumont',
                  when = '2020',
                  row_index = "Race Ethnicity",
                  col_index = 'Tenure Status')

In [ ]:
# look just at site 1
site1_hua_gdf = hua_tfsites_gdf_addrace[hua_tfsites_gdf_addrace['Site_ID'] == 1]
PopResultsTable.pop_results_table(
                  input_df = site1_hua_gdf, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = 'Site 1 - West Port Arthur',
                  when = '2020',
                  row_index = "Race Ethnicity",
                  col_index = 'Tenure Status')

In [ ]:
# look just at site 2
site2_hua_gdf = hua_tfsites_gdf_addrace[hua_tfsites_gdf_addrace['Site_ID'] == 2]
PopResultsTable.pop_results_table(
                  input_df = site2_hua_gdf, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = 'Site 2 - Southeast Beaumont',
                  when = '2020',
                  row_index = "Race Ethnicity",
                  col_index = 'Tenure Status')

In [ ]:
PopResultsTable.pop_results_table(
                  input_df = hua_tfsites_gdf_addrace, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = 'Southeast Texas',
                  when = '2020',
                  row_index = "Household Income Group",
                  col_index = 'Tenure Status')

In [ ]:
# look just at site 1
PopResultsTable.pop_results_table(
                  input_df = site1_hua_gdf, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = 'Site 1 - Southeast Beaumont',
                  when = '2020',
                  row_index = "Household Income Group",
                  col_index = 'Tenure Status')

In [ ]:
# look just at site 2
PopResultsTable.pop_results_table(
                  input_df = site2_hua_gdf, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = 'Site 2 - Southeast Beaumont',
                  when = '2020',
                  row_index = "Household Income Group",
                  col_index = 'Tenure Status')

# Output files

In [ ]:
# Save Work at this point as CSV
savefile = programname+".csv"
savefolder = programname
savefile_path = os.path.join(savefolder, savefile)
hua_with_grid.to_csv(savefile_path, index=False)